# Example 1: Ground Motion Records Processing for Intensity Measure Extraction

## Introduction

This Jupyter Notebook provides a workflow for reading ground motion record files and extracting distinct intensity measure (IM) levels across various IM types. Ground motion intensity measures are essential parameters that characterize the ground shaking and potential impact of seismic records on structures. By extracting these measures, we can better understand the demand imposed by ground motions on structural systems and eventually characterise a probabilistic demand-intensity model to building classes under investigation (Follow Example 2).

The main goals of this notebook are:

1. **Read and Process Ground Motion Records**: Load seismic record files. The files should be in a format that allows easy parsing and processing. In this example, each ground-motion record consists of an acceleration time-history and time-step files.

2. **Calculate the Response Spectrum**

3. **Extract Intensity Measure (IM) Levels**: For each ground motion record, calculate various intensity measures, such as:
   - Peak Ground Acceleration (PGA)
   - Peak Ground Velocity (PGV)
   - Peak Ground Displacement (PGD)
   - Spectral Acceleration (SA) at specified periods (e.g., 0.3s, 0.6s, 1.0s)
   - Average Spectral Acceleration (AvgSA) considering a range of 0.2T to 1.5T at user-specified periods (e.g., 0.3s, 0.6s, 1.0s)
   - Average Spectral Acceleration (AvgSA) considering a user-defined range of periods 
   - Arias Intensity (AI), Cumulative Absolute Velocity (CAV) and Significant Duration (D595)
   - Filtered Incremental Velocity 3 (FIV3)
  
Please note that the class used "IMCalculator" is always subject to updating to reflect state-of-the-art IM inclusion.

4. **Organize and Export Results**: Organize the extracted IM levels for each record, summarizing the results for analysis and potential use in structural modeling and assessment studies. The results are stored into a dictionary type variable and exported into a pickle file. 

This notebook provides a step-by-step guide to process ground motion records efficiently, extract a wide range of IM types, and present the data in an accessible format. Basic knowledge of earthquake engineering and data processing is recommended to fully utilize the material in this notebook.

---

## Import Libraries ## 

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the IMCalculator class
from openquake.vmtk.im_calculator import IMCalculator 
from openquake.vmtk.utilities     import sorted_alphanumeric, export_to_pkl

## Define Plotting Constants ##

In [ ]:
FONTSIZE_1 = 16
FONTSIZE_2 = 14
FONTSIZE_3 = 12

LINEWIDTH_1= 3
LINEWIDTH_2= 2
LINEWIDTH_3 = 1

RESOLUTION = 500

MARKER_SIZE_1 = 100
MARKER_SIZE_2 = 60
MARKER_SIZE_3 = 10

COLOR = "#399283"

## Load Acceleration Time-History ## 

In [ ]:
# The set of ground-motion records utilized correspond to the acceleration time-histories employed in 
# developing the European Seismic Risk Model (2020). This dataset is readily accessible at:  
# https://gitlab.seismo.ethz.ch/efehr/esrm20_vulnerability/-/tree/master/scripts/vmtk/gmrs?ref_type=heads

# Input the intensity measure types required for processing
IMT = ['PGA','PGV','PGD','SA(0.3s)','SA(0.6s)','SA(1.0s)','AvgSA','AvgSA(0.3s)','AvgSA(0.6s)','AvgSA(1.0s)','AI','D595']

# Load the records directory which contains two folders where the time and acceleration arrays of each ground-motion record are stored
gm_directory = './in/records' 

# Fetch the acceleration files
gmrs = sorted_alphanumeric(os.listdir(os.path.join(gm_directory, 'acc')))

# Initialise the storage dictionary and name the "keys" in accordance with the defined IMT
imls = {}
for i, current_imt in enumerate(IMT):    
    imls[current_imt] = []

# Visualise the dictionary
print(imls)

## Process Intensity Measures ## 

In [ ]:
# Loop over the files
for i in range(len(gmrs)):
    
    ### Load the acceleration time-histories and the time-step files
    current_acc = pd.read_csv(os.path.join(gm_directory,'acc',f'acc_{i}.csv'), header=None).to_numpy()
    current_dts = pd.read_csv(os.path.join(gm_directory,'dts',f'dts_{i}.csv'), header=None).to_numpy()
    
    ### Define the time-step of the acceleration time-history
    dt_gm = (current_dts[1] - current_dts[0]).item() # Subtract any consecutive rows of the time-step file

    ### Plot the time history only for the first ground-motion record
    if i==0:
        plt.plot(current_dts, current_acc, color = COLOR, lw=LINEWIDTH_3)
        plt.xlabel('Pseudo-time [s]', fontsize= FONTSIZE_1)
        plt.ylabel('Acceleration [g]', fontsize = FONTSIZE_1)
        plt.grid(visible=True, which='major')
        plt.grid(visible=True, which='minor')
        plt.xlim([0.00, np.max(current_dts)])
        plt.show()

    ### Initialise the Intensity Measure Calculator class
    # The class "IMCalculator" is initialised using the acceleration array and the time-step scalar value
    im_calculator = IMCalculator(current_acc,dt_gm) 

    ### Calculate amplitude-based intensity measures: peak ground acceleration, peak ground velocity, peak ground displacement 
    pga, pgv, pgd = im_calculator.get_amplitude_ims()
    
    ### Calculate duration-based intensity measures: arias intensity, cumulative absolute velocity, significant duration d595
    ai, cav, t595 = im_calculator.get_duration_ims()
     
    ### Calculate and plot the acceleration, velocity and displacement spectra 
    periods, sd, sv, sa = im_calculator.get_spectrum()

    ### Plot the spectra only for the first ground-motion record
    if i==0:

        # Acceleration Spectrum
        plt.plot(periods, sa, color = COLOR, lw=LINEWIDTH_3)
        plt.xlabel('Period [s]', fontsize= FONTSIZE_1)
        plt.ylabel('Pseudo Spectral Acceleration [g]', fontsize = FONTSIZE_1)
        plt.grid(visible=True, which='major')
        plt.grid(visible=True, which='minor')
        plt.xlim([0.00, np.max(periods)])
        plt.show()

        # Velocity Spectrum 
        plt.plot(periods, sv, color = COLOR, lw=LINEWIDTH_3)
        plt.xlabel('Period [s]', fontsize= FONTSIZE_1)
        plt.ylabel('Pseudo Spectral Velocity [m/s]', fontsize = FONTSIZE_1)
        plt.grid(visible=True, which='major')
        plt.grid(visible=True, which='minor')
        plt.xlim([0.00, np.max(periods)])
        plt.show()

        # Displacement Spectrum 
        plt.plot(periods, sd, color = COLOR, lw=LINEWIDTH_3)
        plt.xlabel('Period [s]', fontsize= FONTSIZE_1)
        plt.ylabel('Pseudo Spectral Displacement [m]', fontsize = FONTSIZE_1)
        plt.grid(visible=True, which='major')
        plt.grid(visible=True, which='minor')
        plt.xlim([0.00, np.max(periods)])
        plt.show()

    ### Calculate the spectral acceleration values at distinct periods (e.g., 0.3, 0.6 and 1.0s)
    sa03 = im_calculator.get_sa(0.3)
    sa06 = im_calculator.get_sa(0.6)
    sa10 = im_calculator.get_sa(1.0)

    ### Calculate the average spectral acceleration values at distinct periods (e.g., 0.3, 0.6 and 1.0s) 
    ### with an auto-defined range of 0.2T and 1.5T
    avgsa03 = im_calculator.get_saavg(0.3)
    avgsa06 = im_calculator.get_saavg(0.6)
    avgsa10 = im_calculator.get_saavg(1.0)

    ### Calculate the average spectral acceleration values at distinct periods (e.g., 0.3, 0.6 and 1.0s)
    ### with a user defined range 
    periods_list = np.linspace(0.1,1.0,10)
    avgsa = im_calculator.get_saavg_user_defined(periods_list)

    ### Print outputs only for first record
    if i==0:
        print(f"Peak Ground Acceleration (PGA): {pga:.4f} g²")
        print(f"Peak Ground Velocity (PGV): {pgv:.4f} m/s")
        print(f"Peak Ground Displacement (PGD): {pgd:.4f} m")
    
        print(f"Arias Intensity (AI): {ai:.4f} m/s")
        print(f"Cumulative Absolute Velocity (CAV): {cav:.4f} m/s")
        print(f"Significant Duration (5%-95% AI): {t595:.4f} s")
       
        print(f"Spectral Acceleration (SA at 0.3s): {sa03:.4f} g")
        print(f"Spectral Acceleration (SA at 0.6s): {sa06:.4f} g")
        print(f"Spectral Acceleration (SA at 1.0s): {sa10:.4f} g")
    
        print(f"Average Spectral Acceleration (AvgSA at 0.3s): {avgsa03:.4f} g")
        print(f"Average Spectral Acceleration (AvgSA at 0.6s): {avgsa06:.4f} g")
        print(f"Average Spectral Acceleration (AvgSA at 1.0s): {avgsa10:.4f} g")
        print(f"Average Spectral Acceleration (AvgSA at [0.01-1.0]): {avgsa:.4f} g")

    ### Store the intensity measures in dictionary
    for j, current_imt in enumerate(IMT):

        if current_imt == 'PGA':
            imls[current_imt].append(pga) 
        elif current_imt == 'PGV':
            imls[current_imt].append(pgv) 
        elif current_imt == 'PGD':
            imls[current_imt].append(pgd)         
        elif current_imt == 'SA(0.3s)':
            imls[current_imt].append(sa03) 
        elif current_imt == 'SA(0.6s)':
            imls[current_imt].append(sa06) 
        elif current_imt == 'SA(1.0s)':
            imls[current_imt].append(sa10) 
        elif current_imt == 'AvgSA':
            imls[current_imt].append(avgsa)                                               
        elif current_imt == 'AvgSA(0.3s)':
            imls[current_imt].append(avgsa03) 
        elif current_imt == 'AvgSA(0.6s)':
            imls[current_imt].append(avgsa06) 
        elif current_imt == 'AvgSA(1.0s)':
            imls[current_imt].append(avgsa10)
        elif current_imt == 'AI':
            imls[current_imt].append(ai) 
        elif current_imt == 'AI':
            imls[current_imt].append(cav) 
        elif current_imt == 'D595':
            imls[current_imt].append(t595) 

# Export to pickle format 
export_to_pkl(os.path.join(gm_directory,'imls_esrm20.pkl'), imls)